In [1]:
from functions import *

In [2]:
params = read_parameters()
subfolder = params['subfolder']
file_idx = params['file_idx']
initial_feed_volume_L = params['initial_feed_volume_L']
reset_columns_when_OFF = params['reset_columns_when_OFF']

parameters:
subfolder: UF/
file_idx_uppaal: 3
file_idx: 0
tmp_idx: 0
initial_feed_volume_L: 100
reset_columns_when_OFF: True
drop_initial_final_off_rows: True
drop_off_rows: True
drop_outliers: True
plot_scatterplot_matrix: False
use_default_arima_params: True
default_arima_params: [1, 1, 0]
include_arima_simulations_in_analysis: True


In [3]:
in_folder = PATH_SENSORS_DATA_RAW_UF
out_folder = PATH_SENSORS_DATA_EXT_UF_V1
cur_file, file_path = get_input_file(in_folder=in_folder, in_file_idx=file_idx, log=True)
out_file_path = out_folder + cur_file
print(f"\noutput file:\n{out_file_path}")

file list:
0 /app/data/from_sensors/0_raw/UF/2024-07-11 tap water - reflux valve closed.csv

input file:
/app/data/from_sensors/0_raw/UF/2024-07-11 tap water - reflux valve closed.csv

output file:
/app/data/from_sensors/1_extended/UF/v1/2024-07-11 tap water - reflux valve closed.csv


In [4]:
df_experiments = pd.read_excel(FILE_EXPERIMENTS_METADATA)
df_membranes = pd.read_excel(FILE_MEMBRANES_METADATA)

In [5]:
cur_file_name = re.sub('.csv', '', cur_file)
# filter only the current experiment data
cur_experiment = df_experiments[df_experiments['data file'] == cur_file_name]
check_df_size_after_filter(cur_experiment, FILE_EXPERIMENTS_METADATA, 'experiments')
membrane_model = cur_experiment['membrane model'].values[0]
# filter only the used membrane data
cur_membrane = df_membranes[df_membranes['membrane model'] == membrane_model]
check_df_size_after_filter(cur_membrane, FILE_MEMBRANES_METADATA, 'memebranes')

ERROR: no associated experiments metadata found, please fill the data in the input file /app/data/metadata/experiments.xlsx!


SystemExit: 1

/opt/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
df = pd.read_csv(file_path)
# print([ type(c) for c in df.columns ])
df.rename(columns=UF_COLUMNS, inplace=True)
df['datetime'] = pd.to_datetime(df["date"] + " " + df["time"], format='%Y/%m/%d %H:%M:%S')
df['time span [s]'] = df['datetime'].diff().dt.total_seconds()
df['time span [s]'] = df['time span [s]'].apply(lambda x : 60 if (58 <= x and x <= 62) else x) # 1-2 secs errors are removed
# df['tank liters [L]'] = df['tank liters [%]'] * FEED_TANK_CAPACITY_LITERS / 100
df.loc[0, 'time span [s]'] = 0.0
DATE = df.loc[0, 'datetime'].date().isoformat()
df = df[change_column_index(df.columns.tolist(), 'TMP [bar]', 7)] 
df = df[change_column_index(df.columns.tolist(), 'datetime', 0)] 
df = df[change_column_index(df.columns.tolist(), 'time span [s]', 1)]
# df = df[change_column_index(df.columns.tolist(), 'tank liters [L]', 11)]
df = change_unit_measure(df)
df = df.drop(columns=['date', 'time', 'millisecond [ms]'])

In [11]:
cur_experiment

,data file,experiment date,membrane model,feed concentration g/L,feed concentration measured at row,number of membranes used
5,2024-07-11 clean water - reflux valve closed,2024-07-11,COM0251178,NaN,NaN,3


In [10]:
cur_membrane

,membrane model,usage,material,manufacturer,number of channels,channel dimension [m],diameter [m],length [m],area [m2],pore size [nm],link
1,COM0251178,ultrafiltration,ceramic,LiqTech,30,0.003,0.025,1.178,0.33,60.0,https://liqtech.com/ceramic-membranes-dpf/sili...


In [20]:
num_membrane_used = cur_experiment['number of membranes used'].values[0]
membrane_area = cur_membrane['area [m2]'].values[0]
A = membrane_area * num_membrane_used # m^2
print(f"membrane: {cur_membrane['manufacturer'].values[0]}-{membrane_model}, number of membranes used: {num_membrane_used}, total area: {A} [m^2]")
df['area tot [m^2]'] = A

membrane: LiqTech-COM0251178, number of membranes used: 3, total area: 0.99 [m^2]


In [ ]:
df['prs permeate + ATM [Pa]'] = (df['prs permeate [kPa]'] + PRS_ATM_kpa) * 1000
df['viscosity permeate [Pa s]'] = df.apply((lambda x : calc_viscosity(x['temperature [°C]'], pressure_Pa=x['prs permeate + ATM [Pa]'], element='Water')), axis=1)
df = df[change_column_index(df.columns.tolist(), 'viscosity permeate [Pa s]', 8)]
df = df.drop(columns=['prs permeate + ATM [Pa]'])

In [ ]:
# [L / h]   = [m^2]    * [L / m^2 h]
# flow_prmt = area_tot * flux ==> flux = flow_prmt / area_tot
df['flux [L/m^2h]'] = df['flow permeate [L/h]'] / A

In [88]:
# print("crossflow velocity included!")

# [m/s]        = [m^3 / h]                 / (  [m^2]  * [s/h] )
# vlct_crsflow = liters__to__m3(flow_feed) / (area_tot * SECS_PER_HRS);
# df['vlct crsflow [m/s]'] = (df['flow feed [L/h]'] / (1000.0 * 3600.0)) / A

# flow_rtnt_m3 = 0.25 * area_tot * (SECS_PER_HRS * vlct_crsflow) * mbn_diam / mbn_len
# [m/s]                    =  [m^3 / h]                            / (  [s / h        ) / (   [m^2]  )
#df['vlct crsflow 2 [m/s]'] = (mbn_len * df['flow retentate [L/h]'] / (1000.0 * 3600.0)) / (0.25 * A * (2 * mbn_radius))

In [89]:
# PRS_IN = density [kg / m^3] * gravity [m / s^2] * height [m] = 997 * 9.81 * 1.0 = 9780,57 [Pa] = 9,78 [kPa]
df['prs input [kPa]'] = 10.0 #TODO add this data into machines.xlsx
df = df[change_column_index(df.columns.tolist(), 'prs input [kPa]', 2)]

pressure input included!


In [ ]:
# res_tot = res_mbn + res_conc_pol + res_fouling
# flux = prs_TMP / (viscosity * res_tot) ==> res_tot = prs_TMP / (viscosity_permeate * flux)
# [1/m]             =     [kPa]       / ([m^3 / m^2 s]                              * [kPa s])
df['res tot [1/m]'] = df['TMP [kPa]'] / ((df['flux [L/m^2h]'] / (1000.0 * 3600.0)) * (df['viscosity permeate [Pa s]'] / 1000.0))

In [ ]:
df['viscosity permeate at 20°C [Pa s]'] = df.apply((lambda x : calc_viscosity(20, pressure_Pa=1000*(PRS_ATM_kpa+x['prs permeate [kPa]']), element='Water')), axis=1)
df['flux at 20° [L/m^2h]'] = df['flux [L/m^2h]'] * df['viscosity permeate [Pa s]'] / df['viscosity permeate at 20°C [Pa s]']

In [ ]:
# add bool flag when machine is ON
df['is_ON'] = df['TMP [kPa]'] >= 5

In [ ]:
if reset_columns_when_OFF :
    print(f"setting to zero the following columns when the machine is OFF:")
    for c in df.columns :
        if re.match("^(flow)|(flux)|(res).*", c) :
            print(f" - {c}")
            df[c] = df.apply(reset_cols_if_is_OFF, axis=1, args=('is_ON', c))

setting to zero the following columns when the machine is OFF:
 - flow feed [L/h]
 - flow permeate [L/h]
 - flow retentate [L/h]
 - flux [L/m^2h]
 - res tot [1/m]
 - flux at 20° [L/m^2h]


In [ ]:
df['time [m]'] = df['time span [s]'].cumsum() / 60

In [97]:
# TODO
if False :
    df.loc[0, 'TMP group'] = 0
    for i in range(1, len(df)) :
        cur_group = df.loc[i-1, 'TMP group'] + (0 if df.loc[i, 'changed TMP'] == 0 else 1)
        df.loc[i, 'TMP group'] = cur_group
    print('added TMP groups')

added TMP groups


In [98]:
def get_concentration(df_conc, sample_group=0, initial=1, sample_type='feed') :
    df_tmp = df_conc[(df_conc['sample type'] == sample_type) & (df_conc['initial'] == initial) & (df_conc['sample group'] == sample_group)]
    df_tmp = df_tmp.reset_index()
    conc = np.nan
    if not df_tmp.empty :
        conc = df_tmp.loc[0, 'concentration [g/L]']
    return conc

In [99]:
# by default: only one global
conc_interval = CONCENTRATION_INTERVALS.get(out_file, DEFAULT_CONC_GROUP)
for (k,v) in conc_interval.items() :
    print(f"{k}: {v}")

0: ((None, None), 'unknown')


In [100]:
print("splitting df according to different concentrations")
for (group, ((l,r), title)) in conc_interval.items() :
    dfi = df[l:r].reset_index(drop=False)
    dfi['feed concentration [g/L]'] = np.nan
    dfi['retentate concentration [g/L]'] = np.nan
    dfi['permeate concentration [g/L]'] = np.nan
    FIRST = 0
    LAST  = len(dfi) - 1
    dfi.loc[FIRST,      'feed concentration [g/L]'] = get_concentration(df_conc, sample_group=group, initial=1, sample_type='feed')
    dfi.loc[FIRST, 'retentate concentration [g/L]'] = get_concentration(df_conc, sample_group=group, initial=1, sample_type='retentate')
    dfi.loc[FIRST,  'permeate concentration [g/L]'] = get_concentration(df_conc, sample_group=group, initial=1, sample_type='permeate')
    dfi.loc[LAST,  'retentate concentration [g/L]'] = get_concentration(df_conc, sample_group=group, initial=0, sample_type='retentate')
    dfi.loc[LAST,   'permeate concentration [g/L]'] = get_concentration(df_conc, sample_group=group, initial=0, sample_type='permeate')
    # store initial concentration also as a constant column
    dfi['initial feed concentration [g/L]']         = get_concentration(df_conc, sample_group=group, initial=1, sample_type='feed')
    dfi['initial retentate concentration [g/L]']    = get_concentration(df_conc, sample_group=group, initial=1, sample_type='retentate')
    dfi['initial permeate concentration [g/L]']     = get_concentration(df_conc, sample_group=group, initial=1, sample_type='permeate')
    # start from t=1
    dfi['time [m]'] = dfi['time [m]'] - dfi.loc[0, 'time [m]'] + 1
    out_file_i = (out_folder + DATE + ' ' + str(group) + ' ' + title + '.csv')
    # EXPORT according to concentrations
    dfi.to_csv(out_file_i, index=False)

splitting df according to different concentrations


In [101]:
# start from t=1
# df['time [m]'] = df['time [m]'] - df.loc[0, 'time [m]'] + 1

In [102]:
#############################
# EXPORT
# print(f"extended dataset exported as csv: {out_file}")
# df.to_csv(out_file, index=False)